In [ ]:
import os
import numpy as np
import pickle as pkl
from openai import OpenAI

## Load Dataset

In [ ]:
indicator = 'sp500'

indicator2name = {
    'sp500': 'S&P 500',
    'nikkei': 'Nikkei 225'
}

In [ ]:
with open('indices.pkl', 'rb') as f:
    indices = pkl.load(f)

with open(f'time_series.pkl', 'rb') as f:
    data = pkl.load(f)

In [ ]:
data_size = data.shape[0]
window_size = 20
print(data_size, data.shape, window_size)

In [ ]:
texts = {}
for i in indices:
    with open(os.path.join('gpt_summary', f'{i}.txt'), 'r') as f:
        text = f.read()
        texts[i] = text

In [ ]:
data_size = len(indices)

num_train = int(data_size * 0.6)
num_test = int(data_size * 0.2)
num_vali = data_size - num_train - num_test

idx_train = np.arange(num_train)
idx_valid = np.arange(num_train, num_train + num_vali)
idx_test = np.arange(num_train + num_vali, num_train + num_vali + num_test)

## Prompt GPT 4

In [ ]:
API_KEY = ''

In [ ]:
system_prompt = f"Your job is to act as a professional financial forecaster. You will be given a financial summary of the past 20 market days. Based on this information, your task is to predict whether the {indicator2name[indicator]} price will decrease by more than 1%, increase by more than 1%, or change minimally in the next market day."
print(system_prompt)

In [ ]:
client = OpenAI(api_key=API_KEY)

In [ ]:
for _i in idx_test:
    i = indices[_i]
    
    user_prompt = f"Your task is to predict whether the {indicator2name[indicator]} price will:\n"
    user_prompt += f"(1) Decrease: decrease by more than 1%\n(2) Increase: increase by more than 1%\n(3) Neutral: change minimally, between -1% to 1%\nin the next market day. "
    user_prompt += f"The financial situation of the last {window_size} market days is summarized as follows:\n\n"
    
    _text = texts[i].replace('\n\n', ' ')
    user_prompt += f"{_text}\n\n"
    
    user_prompt += f"Based on this information, predict whether the {indicator2name[indicator]} price will decrease by more than 1%, increase by more than 1%, or otherwise (neutral), in the next market day. "
    user_prompt += f"Respond with either \'decrease\', \'increase\', or \'neutral\'. Do not provide any other details. "
    
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
        {
          "role": "system",
          "content": system_prompt
        },
        {
          "role": "user",
          "content": user_prompt
        }
        ],
        temperature=0.7,
        max_tokens=2048,
        top_p=1
    )

    text = response.choices[0].message.content

    with open(f'gpt_predict_text/{i}_{indicator}.txt', 'w') as f:
        f.write(text)